# Souces and sinks

### Load imports

In [1]:
import cobra
import pandas as pd
import sys

source_dir = '../src'
sys.path.append(source_dir)

### Load the genome scale model

In [2]:
model = cobra.io.json.load_json_model('../genome_scale_models/iYLI647_corr_3.json')

### Keep reaction ids consistent

In [13]:
model.reactions.get_by_id('EX_glc(e)').id  = 'EX_glc_e'
model.reactions.get_by_id('EX_glyc(e)').id = 'EX_glyc_e'
model.reactions.get_by_id('EX_ocdcea(e)').id = 'EX_ocdcea_e'
model.reactions.get_by_id('EX_h2o(e)').id = 'EX_h2o_e'
model.reactions.get_by_id('EX_h(e)').id = 'EX_h_e'
model.reactions.get_by_id('EX_nh4(e)').id = 'EX_nh4_e'
model.reactions.get_by_id('EX_o2(e)').id = 'EX_o2_e'
model.reactions.get_by_id('EX_pi(e)').id = 'EX_pi_e'
model.reactions.get_by_id('EX_so4(e)').id = 'EX_so4_e'

# print an example reaction
model.reactions.get_by_id('EX_glc_e')

Reaction identifier,EX_glc_e
Name,D Glucose exchange
Memory address,0x13d87dba0
Stoichiometry,glc_D[e] <=> D_Glucose <=>
GPR,YALI0D01111g or YALI0D18876g or YALI0D00132g or YALI0B01342g or YALI0E23287g or YALI0B00396g or...
Lower bound,-10.0
Upper bound,1000.0


### Load pFBA and E-Flux2 Data

In [6]:
glucose_df = pd.read_csv('../results/gsm_fluxes/glucose_gsm_fluxes.csv')
glycerol_df = pd.read_csv('../results/gsm_fluxes/glycerol_gsm_fluxes.csv')
oleic_acid_df = pd.read_csv('../results/gsm_fluxes/oleic_acid_gsm_fluxes.csv')
glucose_df

,reaction_id,reaction_name,full_reaction,glucose_pfba_flux,glucose_pfba_LB,glucose_pfba_UB,glucose_1_eflux2_flux,glucose_1_trans_LB,glucose_1_trans_UB,glucose_1_eflux2_LB,...,glucose_2_eflux2_flux,glucose_2_trans_LB,glucose_2_trans_UB,glucose_2_eflux2_LB,glucose_2_eflux2_UB,glucose_3_eflux2_flux,glucose_3_trans_LB,glucose_3_trans_UB,glucose_3_eflux2_LB,glucose_3_eflux2_UB
0,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-542.558865,-1000.000000,-182.367472,-606.412165,-inf,inf,-2678.025975,...,-612.516933,-inf,inf,-4060.379155,-444.256106,-611.897623,-inf,inf,-3827.921541,-439.150238
1,H2Ot,H2O transport via diffusion,h2o[e] <=> h2o[c],-325.018420,-475.756773,-241.338827,-298.691626,-inf,inf,-407.801785,...,-292.224228,-inf,inf,-423.169991,-287.575351,-293.509932,-inf,inf,-413.628786,-286.500906
2,EX_h2o_e,H2O exchange,h2o[e] <=>,325.018420,241.338827,475.756773,298.691626,-8.667672e+04,8.667672e+03,285.669379,...,292.224228,-1.433193e+05,1.433193e+04,287.575351,423.169991,293.509932,-1.414095e+05,1.414095e+04,286.500906,413.628786
3,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],315.568098,-296.592265,1000.000000,279.012358,-2.790124e+02,2.790124e+02,-114.682450,...,337.685599,-5.459032e+02,5.459032e+02,-102.838195,545.903208,337.449074,-4.940848e+02,4.940848e+02,-109.515099,494.084797
4,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],315.568098,184.522645,658.920270,340.938768,0.000000e+00,3.682642e+04,262.017417,...,337.685599,0.000000e+00,5.749684e+04,264.274760,588.805112,337.449074,0.000000e+00,5.824431e+04,259.625182,590.567037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,FA120tp,fatty acid peroxisomal transport,ddca[c] --> ddca[x],0.000000,0.000000,-0.000000,0.000000,0.000000e+00,4.313640e+03,0.000000,...,0.000000,0.000000e+00,6.715082e+03,0.000000,-0.000000,0.000000,0.000000e+00,7.166492e+03,0.000000,-0.000000
1346,FA120ACPtm,fatty acyl ACP mitochondrial transport,ddcaACP[m] --> ddcaACP[c],0.000000,0.000000,-0.000000,0.000000,0.000000e+00,inf,0.000000,...,0.000000,0.000000e+00,inf,0.000000,-0.000000,0.000000,0.000000e+00,inf,0.000000,-0.000000
1347,FA120ACPH,fatty acyl ACP hydrolase,ddcaACP[c] + h2o[c] <=> ACP[c] + ddca[c] + h[c],0.000000,0.000000,-0.000000,0.000000,-5.692927e+03,5.692927e+03,0.000000,...,0.000000,-8.413559e+03,8.413559e+03,0.000000,-0.000000,0.000000,-9.940381e+03,9.940381e+03,0.000000,-0.000000
1348,FA100tp,fatty acid peroxisomal transport,dca[c] --> dca[x],0.000000,0.000000,-0.000000,0.000000,0.000000e+00,4.313640e+03,0.000000,...,0.000000,0.000000e+00,6.715082e+03,0.000000,-0.000000,0.000000,0.000000e+00,7.166492e+03,0.000000,-0.000000


### 

### Define a function to get the sources and sinks of a given metabolite 

In [43]:
# Define a function to get the sources and sinks of a given metabolite 
def get_sources_and_sinks(df, column_name, metabolite_id):
    metabolite = model.metabolites.get_by_id(metabolite_id)

    # make empty source and sink dataframes
    sources = []
    sinks = []

    # loop over rows of the dataframe
    for _, row in df.iterrows():
        reaction_id = row['reaction_id']
        reaction = model.reactions.get_by_id(reaction_id)
        flux = row[column_name]

        # check if the metabolite is in the reaction
        if metabolite not in reaction.metabolites:
            continue
        else:
            metabolite_coefficient = reaction.metabolites[metabolite]
            if flux * metabolite_coefficient > 0:
                sources.append({
                    'reaction_id': reaction_id,
                    'reaction_name': reaction.name,
                    'full_reaction': reaction.reaction,
                    'metabolite_coefficient': metabolite_coefficient,
                    'flux': flux,
                    'total_flux': flux * metabolite_coefficient,
                })
            else:
                sinks.append({
                    'reaction_id': reaction_id,
                    'reaction_name': reaction.name,
                    'full_reaction': reaction.reaction,
                    'metabolite_coefficient': metabolite_coefficient,
                    'flux': flux,
                    'total_flux': flux * metabolite_coefficient,
                })

    source_df = pd.DataFrame(sources)
    sink_df = pd.DataFrame(sinks)

    # sort the dataframes by absolute value of total flux
    source_df = source_df.assign(abs_total_flux=source_df.total_flux.abs())
    source_df = source_df.sort_values('abs_total_flux', ascending=False)

    sink_df = sink_df.assign(abs_total_flux=sink_df.total_flux.abs())
    sink_df = sink_df.sort_values('abs_total_flux', ascending=False)
    


    return source_df, sink_df

sources, sinks = get_sources_and_sinks(glucose_df, 'glucose_pfba_flux', 'nadph[c]')

display(sources)
display(sinks)

,reaction_id,reaction_name,full_reaction,metabolite_coefficient,flux,total_flux,abs_total_flux
0,GND,phosphogluconate dehydrogenase,6pgc[c] + nadp[c] --> co2[c] + nadph[c] + ru5p...,1.0,71.127154,71.127154,71.127154
1,G6PDH2,glucose 6 phosphate dehydrogenase,g6p[c] + nadp[c] --> 6pgl[c] + h[c] + nadph[c],1.0,69.901688,69.901688,69.901688
2,MTHFD,methylenetetrahydrofolate dehydrogenase NADP,mlthf[c] + nadp[c] <=> methf[c] + nadph[c],1.0,2.425342,2.425342,2.425342
3,PPND2,prephenate dehydrogenase NADP,nadp[c] + pphn[c] --> 34hpp[c] + co2[c] + nadp...,1.0,0.590735,0.590735,0.590735


,reaction_id,reaction_name,full_reaction,metabolite_coefficient,flux,total_flux,abs_total_flux
0,GLUDy,glutamate dehydrogenase NADP,glu_L[c] + h2o[c] + nadp[c] <=> akg[c] + h[c] ...,1.0,-52.211097,-52.211097,52.211097
4,FAS80COA_L,fatty acyl CoA synthase n C80CoA lumped reaction,accoa[c] + 9.0 h[c] + 3.0 malcoa[c] + 6.0 nadp...,-6.0,3.674315,-22.045892,22.045892
2,HMGCOAR,Hydroxymethylglutaryl CoA reductase,coa[c] + mev_R[c] + 2.0 nadp[c] <=> 2.0 h[c] +...,2.0,-3.827858,-7.655717,7.655717
3,FAS100COA,fatty acyl CoA synthase n C100CoA,3.0 h[c] + malcoa[c] + 2.0 nadph[c] + occoa[c]...,-2.0,3.674315,-7.348631,7.348631
5,FAS120COA,fatty acyl CoA synthase n C120CoA,dcacoa[c] + 3.0 h[c] + malcoa[c] + 2.0 nadph[c...,-2.0,3.608628,-7.217255,7.217255
...,...,...,...,...,...,...,...
40,CERS2_24,Ceramide 2 synthase 24C,cer1_24[c] + h[c] + nadph[c] + o2[c] --> cer2_...,-1.0,0.000000,-0.000000,0.000000
41,CERS2_26,Ceramide 2 synthase 26C,cer1_26[c] + h[c] + nadph[c] + o2[c] --> cer2_...,-1.0,0.000000,-0.000000,0.000000
42,CERS324,Ceramide 3 synthase 24C,cer2_24[c] + h[c] + nadph[c] + o2[c] --> cer3_...,-1.0,0.000000,-0.000000,0.000000
43,CERS326,Ceramide 3 synthase 26C,cer2_26[c] + h[c] + nadph[c] + o2[c] --> cer3_...,-1.0,0.000000,-0.000000,0.000000


In [41]:
# get sum of absolute values of fluxes for both dfs
sources_sum = sources['total_flux'].abs().sum()
sinks_sum = sinks['total_flux'].abs().sum()

# print the sum of fluxes
print('sources sum: ', sources_sum)
print('sinks sum: ', sinks_sum)

sources sum:  144.04491875990502
sinks sum:  144.044918759903


In [44]:
len(sources)

4